**Table of contents**<a id='toc0_'></a>    
- [Synthetic Data](#toc1_)    
- [View Dataset](#toc2_)    
- [Spectral Clustering](#toc3_)    
- [DBSCAN Clustering](#toc4_)    
- [Xmean Clustering](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
from sklearn.cluster import SpectralClustering, DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import os
from geometric_Xmean import fit

np.random.seed(42)
np.set_printoptions(precision=3, suppress=True, linewidth=200)
rsrc = os.environ["RSRC_DIR"]

# <a id='toc1_'></a>[Synthetic Data](#toc0_)

In [ ]:
dof = 2
kmin = 3
kmax = 40
# weights = [0.2676, 0.3232, 0.2576, 0.0303, 0.0917, 0.0296]
# weights = [0.2676, 0.3232]
# weights effects the centeroid calculation along each dimension
weights = [0.5] * dof

data1 = np.random.multivariate_normal(
    mean=[1.0] * dof,
    cov=np.diag([0.5] * dof),
    size=500,
)
data2 = np.random.multivariate_normal(
    mean=[5.0] * dof,
    cov=np.diag([0.8] * dof),
    size=500,
)
data3 = np.random.multivariate_normal(
    mean=[8.0] * dof,
    cov=np.diag([0.3] * dof),
    size=500,
)
selected_configurations = np.vstack((data1, data2, data3))

# selected_configurations = np.loadtxt(
xmeans = fit(
    selected_configurations[:, 0:dof],
    kmax=kmax,
    kmin=kmin,
    weights=np.array(weights) * dof,
)

N = xmeans.k
labels = xmeans.labels_
center = xmeans.centroid_centres_
points_per_cluster = xmeans.count

print("Number of clusters assigned: %d." % N)
print("Cluster centers:\n", center)
print("labels:\n", labels)
print("Points per cluster:\n", points_per_cluster)

fig, ax = plt.subplots()

scatter = ax.scatter(
    selected_configurations[:, 0],
    selected_configurations[:, 1],
    c=labels,
    cmap="tab20",
    s=5,
)

ax.plot(
    center[:, 0],
    center[:, 1],
    "kx",
    markersize=10,
    markeredgewidth=2,
    label="Centroids",
)
legend1 = ax.legend(
    *scatter.legend_elements(),
    title="Clusters",
    loc="upper right",
)
ax.add_artist(legend1)
plt.xlabel("X1")
plt.ylabel("X2")
plt.title("X-means Clustering Result")
plt.show()

In [ ]:
cspace_dataset = np.load(os.path.join(rsrc, "cspace_dataset.npy"))
cspace_obs = np.load(os.path.join(rsrc, "cspace_obstacles.npy"))
cspace_nearest = np.load(os.path.join(rsrc, "cspace_dataset_nearest_distance.npy"))

# <a id='toc2_'></a>[View Dataset](#toc0_)

In [ ]:
vmax = np.max(cspace_nearest[:, 2])
vmin = np.min(cspace_nearest[:, 2])
d = cspace_nearest[:, 2]

fig, ax = plt.subplots()
sc = ax.scatter(
    cspace_nearest[:, 0],
    cspace_nearest[:, 1],
    c=d,
    cmap="viridis",
    vmin=vmin,
    vmax=vmax,
)
ax.plot(
    cspace_obs[:, 0],
    cspace_obs[:, 1],
    "ro",
    markersize=2,
    label="Obstacle C-space",
    alpha=0.3,
)
plt.colorbar(sc, label="Nearest Distance to Obstacle")
ax.set_xlabel("Theta 1")
ax.set_ylabel("Theta 2")
ax.set_aspect("equal", adjustable="box")
plt.show()

# <a id='toc3_'></a>[Spectral Clustering](#toc0_)

In [ ]:
clustering = SpectralClustering(
    n_clusters=4,
    eigen_solver="arpack",
    affinity="nearest_neighbors",
    random_state=42,
)
clustering.fit(cspace_obs)
labels = clustering.labels_

fig, ax = plt.subplots()
sc = ax.scatter(
    cspace_obs[:, 0],
    cspace_obs[:, 1],
    c=labels,
    cmap="viridis",
)
ax.set_title("Spectral Clustering of C-space Obstacles")
ax.set_aspect("equal", "box")
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)
ax.grid(True)
plt.show()

# <a id='toc4_'></a>[DBSCAN Clustering](#toc0_)

In [ ]:
clustering = DBSCAN(eps=0.5, min_samples=5)
clustering.fit(cspace_obs)
labels = clustering.labels_

fig, ax = plt.subplots()
sc = ax.scatter(
    cspace_obs[:, 0],
    cspace_obs[:, 1],
    c=labels,
    cmap="viridis",
)
ax.set_title("DBSCAN Clustering of C-space Obstacles")
ax.set_aspect("equal", "box")
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)
ax.grid(True)
plt.show()

# <a id='toc5_'></a>[Xmean Clustering](#toc0_)

In [ ]:
dof = 2
kmin = 3
kmax = 40
weights = [0.5] * dof

xmeans = fit(cspace_obs[:, 0:dof], kmin=kmin, kmax=kmax, weights=weights)
N = xmeans.k
labels = xmeans.labels_
center = xmeans.centroid_centres_
points_per_cluster = xmeans.count

print("Number of clusters assigned: %d." % N)
print("Cluster centers:\n", center)
print("labels:\n", labels)
print("Points per cluster:\n", points_per_cluster)

group1_point = cspace_obs[labels == 0]
group2_point = cspace_obs[labels == 1]
group3_point = cspace_obs[labels == 2]

fig, ax = plt.subplots()
ax.plot(
    center[:, 0],
    center[:, 1],
    "kx",
    markersize=10,
    markeredgewidth=2,
    label="Centroids",
)
ax.plot(
    group1_point[:, 0],
    group1_point[:, 1],
    "ro",
    markersize=2,
    label="Cluster 1",
)
ax.plot(
    group2_point[:, 0],
    group2_point[:, 1],
    "go",
    markersize=2,
    label="Cluster 2",
)
ax.plot(
    group3_point[:, 0],
    group3_point[:, 1],
    "bo",
    markersize=2,
    label="Cluster 3",
)
ax.set_xlabel("Theta 1")
ax.set_ylabel("Theta 2")
ax.legend()
plt.show()